In [0]:
# generate_daily_positions.py
import random
from datetime import datetime, timedelta
import pandas as pd
import json
import pytz

def generate_daily_positions(date, currency_pairs):
    """
    Simula posições diárias com:
    - Random walks (posições não mudam drasticamente dia-a-dia)
    - Bias por currency (EUR/USD = posições maiores)
    - Reversões ocasionais (long → short)
    """
    
    positions = []
    
    for pair in currency_pairs:
        # Tamanho base por currency pair
        base_sizes = {
            'EUR/USD': 15_000_000,  # Maior liquidez
            'GBP/USD': 8_000_000,
            'USD/JPY': 10_000_000,
            'USD/CHF': 5_000_000,
            'USD/BRL': 3_000_000    # Menor liquidez
        }
        
        base_size = base_sizes.get(pair, 5_000_000)
        
        # Random walk: +/- 20% do tamanho base
        daily_change = random.uniform(-0.2, 0.2)
        position_size = base_size * (1 + daily_change)
        
        # 70% chance LONG, 30% chance SHORT (bancos geralmente net long)
        direction = 'LONG' if random.random() < 0.7 else 'SHORT'
        
        # 5% chance de "zero position" (flat)
        if random.random() < 0.05:
            position_size = 0
            direction = 'FLAT'
        
        positions.append({
            'date': date,
            'currency_pair': pair,
            'position_size': round(position_size, 2),
            'direction': direction,
            'desk': 'FX Spot Desk',  # Simplificado
            'generated_at': datetime.now()
        })
    
    return pd.DataFrame(positions)

currency_pairs = [
    'EURUSD',
    'GBPUSD',
    'USDJPY',
    'USDCHF',
    'USDBRL'
]

positions_today = generate_daily_positions(
    datetime.now(pytz.timezone("Europe/Madrid")).date(),
    currency_pairs
)

# turn rows into a list[dict]
records = positions_today.to_dict(orient="records")

# serialize to JSON lines (one obj per line) or normal JSON array; pick what you want
json_lines = "\n".join(json.dumps(r, default=str) for r in records)

output_path = "/Volumes/tabular/dataexpert/guidotognini/market_risk/raw_positions"
file_name = f"{datetime.now().date()}_positions.json"
full_path = f"{output_path}/{file_name}"

# write directly to the Volume using dbutils.fs.put
dbutils.fs.put(full_path, json_lines, overwrite=True)